In [11]:
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import shutil

# Convert Masks to YOLO Annotations
### Dataset Format for YOLO
YOLO requires annotations in .txt files (one per image) with bounding boxes or segmentation masks. For object detection (bounding boxes), use this format:
```bash
0 0.5 0.5 0.2 0.3  # class_id, x_center, y_center, width, height
1 0.7 0.3 0.1 0.1
```
If your masks are binary images, use OpenCV to extract bounding boxes:

In [6]:
def convert_mask_to_yolo(mask_path, output_path, class_id=0, mode='segmentation'):
    """
    Convert a mask image to YOLO format annotations.
    Args:
        mask_path: Path to the mask image
        output_path: Path to save the YOLO annotation
        class_id: Class ID for the annotation (default 0)
        mode: 'segmentation' or 'detection' (default 'segmentation')
    """
    # Read the mask image
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Could not read mask: {mask_path}")
        return
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get image dimensions for normalization
    height, width = mask.shape
    
    with open(output_path, 'w') as file:
        for contour in contours:
            if mode == 'segmentation':
                # Handle segmentation mode
                points = contour.squeeze()
                if len(points.shape) < 2:  # Skip invalid contours
                    continue
                
                # Normalize coordinates
                normalized_points = points / [width, height]
                
                # Write points to file
                points_str = " ".join(f"{x:.5f} {y:.5f}" for x, y in normalized_points)
                file.write(f"{class_id} {points_str}\n")
                
            else:  # detection mode
                # Handle detection mode (bounding box)
                x, y, w, h = cv2.boundingRect(contour)
                
                # Convert to YOLO format (center coordinates + width + height)
                center_x = (x + w/2) / width
                center_y = (y + h/2) / height
                norm_width = w / width
                norm_height = h / height
                
                # Write bounding box to file
                file.write(f"{class_id} {center_x:.5f} {center_y:.5f} "
                          f"{norm_width:.5f} {norm_height:.5f}\n")


def process_dataset(base_path):
    """
    Process the entire dataset, organizing images and generating YOLO annotations.
    Args:
        base_path: Base path to the dataset
    """
    # Create a new base folder for processed data
    processed_base_path = Path(base_path) / 'processed_data'
    processed_base_path.mkdir(parents=True, exist_ok=True)

    # Process each split (train, valid, test)
    for split in ['train', 'valid', 'test']:
        # Setup directories in the new base folder
        split_path = processed_base_path / split
        images_dir = split_path / 'images'
        labels_dir = split_path / 'labels'
        
        # Create directories if they don't exist
        images_dir.mkdir(parents=True, exist_ok=True)
        labels_dir.mkdir(parents=True, exist_ok=True)
        
        # Process all mask files in the original dataset
        original_split_path = Path(base_path) / split
        for mask_file in original_split_path.glob('*_mask.png'):
            # Get corresponding image name
            image_name = mask_file.name.replace('_mask.png', '.jpg')
            image_path = mask_file.with_name(image_name)
            
            try:
                # Move image to images directory
                shutil.copy(image_path, images_dir / image_name)
                
                # Create YOLO annotation
                label_name = mask_file.stem.replace('_mask', '') + '.txt'
                label_path = labels_dir / label_name
                
                # Convert mask to YOLO format
                convert_mask_to_yolo(mask_file, label_path)
                
            except FileNotFoundError:
                print(f"Missing image file: {image_path}")
                continue

In [7]:

# Usage example
if __name__ == "__main__":
    dataset_path = Path('../Road_mask/')
    process_dataset(dataset_path)

# Segmentation (YOLOv8)

Install YOLOv8:

In [ ]:
!pip3 install ultralytics

## Training Code

In [ ]:
DATA_YAML_PATH = './data.yaml'


In [ ]:
model = YOLO('yolov8n-seg.pt')  # Segmentation model

model.train(data=DATA_YAML_PATH, project="./Models", epochs=50, imgsz=640, batch=16)

## Testing

In [12]:
TRAINED_MODEL_PATH = './Models/train/train2/weights/best.pt'

TEST_IMAGE_PATH = "../Road_mask_bkp/test/tile_12_jpg.rf.b0d051c082e97538bdd45a416fa6d93b.jpg"

In [ ]:
model = YOLO(TRAINED_MODEL_PATH)
# Predict on a single image
image_path = TEST_IMAGE_PATH
results = model.predict(image_path, imgsz=640, conf=0.5)


# Visualize results
for result in results:
    plotted = result.plot()
    cv2.imshow("Prediction", plotted)
    
    # Wait for the user to press any key to close the window
    if cv2.waitKey(0) & 0xFF == ord('q'):  # Press 'q' to quit
        break
    
    cv2.destroyAllWindows()

# Close the window properly after exiting the loop
cv2.destroyAllWindows()